In [ ]:
import pandas as pd

In [ ]:
# Load dataframes from files exported from Google Sheets

reg = pd.read_excel(r'registrationsheet.xlsx')
con = pd.read_excel(r'confirmationsheet.xlsx')
tra = pd.read_csv(r'trackingsheet.csv')

In [ ]:
# Little bit of cleaning of data


# Remove content from '?' in the social media profile link
tra['Your social media profile link'] = tra['Your social media profile link'].str.split('?').str[0]

# Add 'https://' if missing
tra['Your social media profile link'] = tra['Your social media profile link'].apply(
    lambda x: x if x.startswith('https://') else f'https://{x.lstrip("https://")}'
)

# Remove slashes and spaces from last part of the link
tra['Your social media profile link'] = tra['Your social media profile link'].str.rstrip('/ ').str.lstrip()

# Replace 'https://www.' with 'https://'
tra['Your social media profile link'] = tra['Your social media profile link'].str.replace('https://www.', 'https://', regex=False)

# Clean 'Post link'
tra['Post link'] = tra['Post link'].str.split('?').str[0]  # Remove content after '?'
tra['Post link'] = tra['Post link'].str.strip('"\'/')  # Remove leading/trailing spaces and quotes/slashes

# Format 'Your Email' to lowercase
tra['Your Email'] = tra['Your Email'].str.lower()

reg['Email Address'] = reg['Email Address'].str.lower()

# Convert email addresses to lowercase
con['Email Address'] = con['Email Address'].str.lower()
con['Your Email'] = con['Your Email'].str.lower()

# Clean 'Your profile link'
con['Your profile link'] = con['Your profile link'].str.split('?').str[0]  # Remove content after '?'

# Add 'https://' if missing and clean up spaces/characters
con['Your profile link'] = con['Your profile link'].apply(
    lambda x: 'https://' + x.lstrip(' https://') if isinstance(x, str) and 'https://' not in x else x.strip(' /') if isinstance(x, str) else x
)

# Replace 'https://www.' with 'https://'
con['Your profile link'] = con['Your profile link'].str.replace('https://www.', 'https://', regex=False)



In [ ]:
# Function to get the preferred email
def get_preferred_email(row):
    return row['Your Email'] if 'Your Email' in row and pd.notna(row['Your Email']) else row['Email Address']

# Function to validate and convert dates
def validate_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='coerce').strftime('%m/%d/%Y')
    except (ValueError, OverflowError):
        return None

# Function to validate profile link format
def is_valid_profile_link(link):
    if isinstance(link, str) and link.strip():
        return (link.startswith('https://x.com/') or
                link.startswith('https://linkedin.com/in/') or
                link.startswith('https://tiktok.com/@') or
                link.startswith('https://instagram.com/') or
                link.startswith('https://facebook.com/'))
    return False

# Create a list of unique email addresses from all dataframes using the preferred email column
con['Preferred Email'] = con.apply(get_preferred_email, axis=1)
reg['Preferred Email'] = reg.apply(get_preferred_email, axis=1)
tra['Preferred Email'] = tra.apply(get_preferred_email, axis=1)

# Initialize the final dataframe with required columns
date_range = pd.date_range(start="2024-10-03", end="2024-11-03").strftime('%m/%d/%Y')
columns = ['Name', 'Profile Link', 'Collaborator', 'Days'] + list(date_range)
final = pd.DataFrame(columns=columns)

# Combine all preferred emails into a dictionary to store related information
email_info = {}

# Helper function to update email info
def update_email_info(email, row):
    if email not in email_info:
        email_info[email] = {
            'Name': row.get('Full Name', None),
            'Profile Link': row.get('Your profile link', None),
            'Collaborator': row.get('If yes, from which collaborator did you learn about Learning Utsav?', None),
            'College': row.get('College Name', None),
            'Days': 0,
            'Post Links': {}
        }

# Iterate over con and reg to populate email_info
for df in [con, reg]:
    for _, row in df.iterrows():
        email_1 = row['Email Address']
        email_2 = row['Your Email'] if 'Your Email' in row else None  # Check if 'Your Email' exists
        
        # Update info for both possible emails
        if pd.notna(email_1):
            update_email_info(email_1, row)
        if pd.notna(email_2):
            update_email_info(email_2, row)

# Aggregate post links from tra and check profile links
for _, tra_row in tra.iterrows():
    email = tra_row['Preferred Email']
    post_date = validate_date(tra_row['Social Media Post Date'])
    tra_profile_link = tra_row.get('Your social media profile link', None)
    
    if post_date and post_date in date_range:
        # Check if the email exists in con or reg (email_info)
        if email in email_info:
            # Validate and update profile link if necessary
            current_profile_link = email_info[email]['Profile Link']
            if not is_valid_profile_link(current_profile_link) and is_valid_profile_link(tra_profile_link):
                email_info[email]['Profile Link'] = tra_profile_link
            
            # Update post links, only count unique dates
            if post_date not in email_info[email]['Post Links']:
                email_info[email]['Post Links'][post_date] = tra_row['Post link']
                email_info[email]['Days'] += 1

# Create final dataframe rows only for individuals with at least one post
rows = []
for email, info in email_info.items():
    if info['Days'] > 0:  # Only include if there is at least one post
        row = {
            'Name': info['Name'],
            'Email': email,
            'Profile Link': info['Profile Link'],
            'Collaborator': info['Collaborator'],
            'Days': info['Days'],
            'College': info['College']
        }
        # Add post links to the row for each date in the date range
        for date in date_range:
            row[date] = info['Post Links'].get(date, None)
        rows.append(row)

# Create final dataframe using pd.DataFrame
final = pd.DataFrame(rows)

# Export the DataFrame to CSV without the index
final.to_csv('reward_winners.csv', index=False)

# Show final dataframe
print(final)
